In [1]:
import pandas as pd
import numpy as np

df = pd.read_csv('alldata_date.csv')
people = pd.read_csv("202312_202312_연령별인구현황_연간.csv", encoding="cp949")

In [2]:
people = people.drop(['2023년_거주자_총인구수', '2023년_거주자_연령구간인구수'], axis=1)

In [3]:
people = people.rename(columns={'2023년_거주자_20~29세': '20대',
                                 '2023년_거주자_30~39세': '30대',
                                 '2023년_거주자_40~49세': '40대',
                                 '2023년_거주자_50~59세': '50대',
                                 '2023년_거주자_60~69세': '60대',
                                 })
for i in range(18):
    people = people.rename(index={i: people['행정구역'][i][0:-14]})

In [4]:
people = people.rename(index={'강원특별자치도':'강원도'})

In [5]:
people = people.drop('행정구역', axis=1)

In [7]:
df = df.drop(['Unnamed: 0',
            #   'EXAMIN_BEGIN_DE',
            #  '사용목적1순위값',
            #  '사용목적2순위값',
            #  '평일일평균레저시간값',
            #  '주말일평균레저시간값',
            #  '휴식오락사용비율',
            #  '취미사용비율',
            #  '본인계발사용비율',
            #  '대인관계교제사용비율',
            #  '기타사용비율',
             ],
             axis=1)
df.head()

# 위도경도 , 소득 숫자로, TF-IDF (목적순위) 날짜를 월/일

,EXAMIN_BEGIN_DE,성별,연령대,거주지,가구소득정도,사용목적1순위값,사용목적2순위값,평일일평균레저시간값,주말일평균레저시간값,휴식오락사용비율,취미사용비율,본인계발사용비율,대인관계교제사용비율,기타사용비율,관심레저활동1순위값
0,20230503,M,30대,서울특별시,300이상500만원 미만,스트레스 해소를 위해,가족-지인 등과 시간을 보내기 위해,2,3,30.0,40.0,20.0,10.0,0.0,골프 직접 하기
1,20230503,F,40대,서울특별시,700만원 이상,대인 관계-교제를 위해,마음의 안정-휴식을 위해,1,3,0.0,0.0,50.0,50.0,0.0,영상 컨텐츠 시청
2,20230503,F,50대,서울특별시,500이상700만원 미만,가족-지인 등과 시간을 보내기 위해,대인 관계-교제를 위해,10,12,80.0,0.0,0.0,20.0,0.0,쇼핑
3,20230503,M,40대,서울특별시,700만원 이상,가족-지인 등과 시간을 보내기 위해,마음의 안정-휴식을 위해,1,5,40.0,20.0,20.0,20.0,0.0,보드게임-퍼즐-바둑-장기-체스 등
4,20230503,F,60대,제주특별자치도,700만원 이상,대인 관계-교제를 위해,남는 시간을 보내기 위해,10,10,70.0,10.0,10.0,5.0,5.0,쇼핑


In [8]:
df['계절'] = df['EXAMIN_BEGIN_DE'].apply(lambda x : str(x)[4:6])
type(df['계절'][0])

str

In [9]:
df['계절'] = df['계절'].replace('01', '겨울')
df['계절'] = df['계절'].replace('02', '봄')
df['계절'] = df['계절'].replace('03', '봄')
df['계절'] = df['계절'].replace('04', '봄')
df['계절'] = df['계절'].replace('05', '여름')
df['계절'] = df['계절'].replace('06', '여름')
df['계절'] = df['계절'].replace('07', '여름')
df['계절'] = df['계절'].replace('08', '가을')
df['계절'] = df['계절'].replace('09', '가을')
df['계절'] = df['계절'].replace('10', '가을')
df['계절'] = df['계절'].replace('11', '겨울')
df['계절'] = df['계절'].replace('12', '겨울')

In [10]:
df = df.drop('EXAMIN_BEGIN_DE', axis=1)

In [11]:
# 위경도 데이터 (예시)
location_data = {
    '서울특별시': (37.5665, 126.9780),
    '제주특별자치도': (33.4996, 126.5312),
    '부산광역시': (35.1796, 129.0756),
    '경기도': (37.4138, 127.5183),
    '인천광역시': (37.4563, 126.7052),
    '광주광역시': (35.1595, 126.8526),
    '강원도': (37.8228, 128.1555),
    '전라남도': (34.8160, 126.4629),
    '충청남도': (36.5184, 126.8000),
    '충청북도': (36.6357, 127.4913),
    '경상북도': (36.4919, 128.8889),
    '경상남도': (35.4606, 128.2132),
    '전라북도': (35.7175, 127.1530),
    '울산광역시': (35.5384, 129.3114),
    '대구광역시': (35.8714, 128.6014),
    '대전광역시': (36.3504, 127.3845),
    '세종특별자치시': (36.4800, 127.2890)
}

def get_lat_lon(location):
    return location_data.get(location, (None, None))

# DataFrame에 적용
df['위도'], df['경도'] = zip(*df['거주지'].apply(get_lat_lon))

In [12]:
results = []
또래인구수_list = []  # 새로운 리스트 생성

for _, row in df.iterrows():
    try:
        value = people.loc[row['거주지'], row['연령대']]
        results.append((row['거주지'], row['연령대'], value))
        또래인구수_list.append(value)  # 값을 새 리스트에 추가
    except KeyError:
        # 해당 거주지나 연령대가 people DataFrame에 없는 경우
        results.append((row['거주지'], row['연령대'], None))
        또래인구수_list.append(None)  # None 값을 새 리스트에 추가

# 새로운 열 '또래인구수' 추가
df['또래인구수'] = 또래인구수_list

df.head(3)  # 새로운 열이 추가된 df 확인

,성별,연령대,거주지,가구소득정도,사용목적1순위값,사용목적2순위값,평일일평균레저시간값,주말일평균레저시간값,휴식오락사용비율,취미사용비율,본인계발사용비율,대인관계교제사용비율,기타사용비율,관심레저활동1순위값,계절,위도,경도,또래인구수
0,M,30대,서울특별시,300이상500만원 미만,스트레스 해소를 위해,가족-지인 등과 시간을 보내기 위해,2,3,30.0,40.0,20.0,10.0,0.0,골프 직접 하기,여름,37.5665,126.978,"1,419,153"
1,F,40대,서울특별시,700만원 이상,대인 관계-교제를 위해,마음의 안정-휴식을 위해,1,3,0.0,0.0,50.0,50.0,0.0,영상 컨텐츠 시청,여름,37.5665,126.978,"1,406,016"
2,F,50대,서울특별시,500이상700만원 미만,가족-지인 등과 시간을 보내기 위해,대인 관계-교제를 위해,10,12,80.0,0.0,0.0,20.0,0.0,쇼핑,여름,37.5665,126.978,"1,476,224"


In [13]:
df['또래인구수'] = df['또래인구수'].str.replace(",", "").astype(int)

In [14]:
df['또래인구수'].isnull().sum()

0

In [15]:
df = df.drop('거주지', axis=1)

In [16]:
df = df.replace('무응답', 0)
df = df.replace('300만원 미만', 1)
df = df.replace('300이상500만원 미만', 2)
df = df.replace('500이상700만원 미만', 3)
df = df.replace('700만원 이상', 4)

In [6]:
# # 항목합치기

# friend = ['SNS -인터넷 커뮤니티 활동',
#         '가족-친지 만남',
#         '계모임-동창회-사교모임',
#         '기타 사회교류',
#         '친구-이성친구 만남',
#         '수다-잡담하기',
#         '종교활동',
#         '음주-유흥',
#         ]
# replace_dict = {item: '사교활동' for item in friend}
# df = df.replace(replace_dict)

# game= '''갬블
# 게임
# 보드게임-퍼즐-바둑-장기-체스 등
# e스포츠 영상 시청
# e스포츠 현장 관람'''
# games = game.split("\n")
# df = df.replace(games, "게임")


# sport = '''걷기-속보-조깅 직접 하기
# 구기스포츠 직접 하기
# 기타 스포츠-운동 직접하기
# 등산 직접 하기
# 라켓스포츠 직접 하기
# 무도-격투기 직접 하기
# 수상레저스포츠 직접 하기
# 수영 직접 하기
# 줄넘기-맨손체조-스트레칭 직접 하기
# 볼링_ 당구-포켓볼 직접 하기
# 요가-필라테스 직접 하기
# 익스트림스포츠 직접하기
# 자전거-싸이클-산악자전거 직접 타기
# 헬스-보디빌딩 직접 하기
# 겨울레저스포츠 직접 하기
# 골프 직접 하기
# 댄스스포츠 직접 하기
# 낚시'''
# sports = sport.split("\n")
# df = df.replace(sports, "운동")


# sports_watching = '''격투 스포츠 영상 시청
# 격투 스포츠 현장 관람
# 국내 프로스포츠 영상 시청
# 국내 프로스포츠 현장 관람
# 기타 스포츠 영상-현장 관람
# 골프 현장 관람
# 골프 영상 시청
# 해외 프로스포츠 영상 시청'''
# sports_watchings = sports_watching.split("\n")
# df = df.replace(sports_watchings, "스포츠 관람")


# trip = '''해외여행
# 국내 여행
# 기타 관광-여행
# 캠핑-차박
# 소풍-피크닉
# 맛집-카페 탐방
# 테마파크-시설 방문'''
# trips = trip.split("\n")
# df = df.replace(trips, "여행")


# culture = '''기타 문화예술관람
# 무용 공연 관람
# 문학 행사 관람
# 미술관-전시관 관람
# 음악 공연 관람
# 박물관 관람
# 연극-뮤지컬 관람
# 영화관 관람'''
# cultures = culture.split('\n')
# df = df.replace(cultures, "문화 예술 관람")

# reading = '''독서
# 만화책 보기
# 신문-잡지보기'''
# reading_books = reading.split('\n')
# df = df.replace(reading_books, "독서")

# tired = '''기타 취미-오락-휴식
# 낮잠자기
# 산책-걷기
# 온천-사우나-찜질방
# 드라이브
# 반려동물 돌보기'''
# tired_ = tired.split('\n')
# df = df.replace(tired_, "휴식")

# doing_culture = '''기타 문화예술 직접 활동 및 수강하기
# 춤-무용 직접 활동 및 수강하기
# 문학 직접 활동 및 수강하기
# 미술 직접 활동 및 수강하기
# 전통예술 직접 활동 및 수강하기
# 음악 직접 활동 및 수강하기
# 영상 컨텐츠 제작
# 사진 직접 활동 및 수강하기
# 생활공예-원예
# 인테리어'''
# doing_cultures = doing_culture.split('\n')
# df = df.replace(doing_cultures, "문화 예술 활동")


# self = '''기타 자기계발-자기관리
# 어학-기술-자격증 취득
# 미용'''
# selfs = self.split('\n')
# df = df.replace(selfs, "자기계발")


# exprience = '''축제 참가
# 액티비티-체험'''
# expriences = exprience.split('\n')
# df = df.replace(expriences, "체험 활동")

In [17]:
len(df['관심레저활동1순위값'].unique())

85

In [18]:
df.head()

,성별,연령대,가구소득정도,사용목적1순위값,사용목적2순위값,평일일평균레저시간값,주말일평균레저시간값,휴식오락사용비율,취미사용비율,본인계발사용비율,대인관계교제사용비율,기타사용비율,관심레저활동1순위값,계절,위도,경도,또래인구수
0,M,30대,2,스트레스 해소를 위해,가족-지인 등과 시간을 보내기 위해,2,3,30.0,40.0,20.0,10.0,0.0,골프 직접 하기,여름,37.5665,126.9780,1419153
1,F,40대,4,대인 관계-교제를 위해,마음의 안정-휴식을 위해,1,3,0.0,0.0,50.0,50.0,0.0,영상 컨텐츠 시청,여름,37.5665,126.9780,1406016
2,F,50대,3,가족-지인 등과 시간을 보내기 위해,대인 관계-교제를 위해,10,12,80.0,0.0,0.0,20.0,0.0,쇼핑,여름,37.5665,126.9780,1476224
3,M,40대,4,가족-지인 등과 시간을 보내기 위해,마음의 안정-휴식을 위해,1,5,40.0,20.0,20.0,20.0,0.0,보드게임-퍼즐-바둑-장기-체스 등,여름,37.5665,126.9780,1406016
4,F,60대,4,대인 관계-교제를 위해,남는 시간을 보내기 위해,10,10,70.0,10.0,10.0,5.0,5.0,쇼핑,여름,33.4996,126.5312,93144


In [19]:
# 레이블 인코딩

from sklearn.preprocessing import LabelEncoder
labelcol = ['성별', 
            '연령대' , 
            # '거주지', 
            '사용목적1순위값', 
            '사용목적2순위값', 
            '계절',
            # 'EXAMIN_BEGIN_DE',
            '관심레저활동1순위값',
            ]
le = LabelEncoder()
for column in labelcol:
    df[column] = le.fit_transform(df[column])

# print(df)
original_labels = le.classes_
encoded_labels = le.transform(le.classes_)
mapping = dict(zip(original_labels, encoded_labels))
print("\n취미 매핑:", mapping)
df.head(3)


취미 매핑: {'SNS -인터넷 커뮤니티 활동': 0, 'e스포츠 영상 시청': 1, 'e스포츠 현장 관람': 2, '가족-친지 만남': 3, '갬블': 4, '걷기-속보-조깅 직접 하기': 5, '게임': 6, '겨울레저스포츠 직접 하기': 7, '격투 스포츠 영상 시청': 8, '격투 스포츠 현장 관람': 9, '계모임-동창회-사교모임': 10, '골프 영상 시청': 11, '골프 직접 하기': 12, '골프 현장 관람': 13, '구기스포츠 직접 하기': 14, '국내 여행': 15, '국내 프로스포츠 영상 시청': 16, '국내 프로스포츠 현장 관람': 17, '기타 관광-여행': 18, '기타 문화예술 직접 활동 및 수강하기': 19, '기타 문화예술관람': 20, '기타 사회교류': 21, '기타 스포츠 영상-현장 관람': 22, '기타 스포츠-운동 직접하기': 23, '기타 자기계발-자기관리': 24, '기타 취미-오락-휴식': 25, '낚시': 26, '낮잠자기': 27, '댄스스포츠 직접 하기': 28, '독서': 29, '드라이브': 30, '등산 직접 하기': 31, '라켓스포츠 직접 하기': 32, '만화책 보기': 33, '맛집-카페 탐방': 34, '무도-격투기 직접 하기': 35, '무용 공연 관람': 36, '문학 직접 활동 및 수강하기': 37, '문학 행사 관람': 38, '미술 직접 활동 및 수강하기': 39, '미술관-전시관 관람': 40, '미용': 41, '박물관 관람': 42, '반려동물 돌보기': 43, '보드게임-퍼즐-바둑-장기-체스 등': 44, '볼링_ 당구-포켓볼 직접 하기': 45, '사진 직접 활동 및 수강하기': 46, '사진 촬영': 47, '사회봉사활동': 48, '산책-걷기': 49, '생활공예-원예': 50, '소풍-피크닉': 51, '쇼핑': 52, '수다-잡담하기': 53, '수상레저스포츠 직접 하기': 54, '수영 직접 하기': 55, '수집': 56, '신문-잡지보기': 57, '액티비티

,성별,연령대,가구소득정도,사용목적1순위값,사용목적2순위값,평일일평균레저시간값,주말일평균레저시간값,휴식오락사용비율,취미사용비율,본인계발사용비율,대인관계교제사용비율,기타사용비율,관심레저활동1순위값,계절,위도,경도,또래인구수
0,1,1,2,6,0,2,3,30.0,40.0,20.0,10.0,0.0,12,3,37.5665,126.978,1419153
1,0,2,4,4,5,1,3,0.0,0.0,50.0,50.0,0.0,61,3,37.5665,126.978,1406016
2,0,3,3,0,4,10,12,80.0,0.0,0.0,20.0,0.0,52,3,37.5665,126.978,1476224


In [20]:
df['관심레저활동1순위값'].value_counts()

관심레저활동1순위값
61    3047
49    2222
5     1797
6     1688
15    1371
      ... 
8       24
2       21
9       18
20      13
13      12
Name: count, Length: 85, dtype: int64

In [21]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, accuracy_score

In [22]:
from imblearn.over_sampling import RandomOverSampler
from collections import Counter

# 데이터와 레이블 분리
X = df.drop('관심레저활동1순위값', axis=1)
y = df['관심레저활동1순위값']

# 오버샘플링
ros = RandomOverSampler(random_state=42)
X_resampled, y_resampled = ros.fit_resample(X, y)

# 결과 확인
X_train, X_test, y_train, y_test = train_test_split(X_resampled, y_resampled, test_size=0.2, random_state=11)
Counter(y_resampled)

Counter({12: 3047,
         61: 3047,
         52: 3047,
         44: 3047,
         66: 3047,
         58: 3047,
         77: 3047,
         49: 3047,
         48: 3047,
         15: 3047,
         31: 3047,
         62: 3047,
         75: 3047,
         84: 3047,
         59: 3047,
         53: 3047,
         79: 3047,
         16: 3047,
         29: 3047,
         6: 3047,
         80: 3047,
         63: 3047,
         40: 3047,
         50: 3047,
         5: 3047,
         14: 3047,
         32: 3047,
         3: 3047,
         83: 3047,
         39: 3047,
         10: 3047,
         25: 3047,
         24: 3047,
         76: 3047,
         17: 3047,
         55: 3047,
         73: 3047,
         0: 3047,
         69: 3047,
         30: 3047,
         27: 3047,
         34: 3047,
         78: 3047,
         65: 3047,
         81: 3047,
         70: 3047,
         68: 3047,
         22: 3047,
         51: 3047,
         18: 3047,
         82: 3047,
         43: 3047,
         4: 3047

In [24]:
X_train

,성별,연령대,가구소득정도,사용목적1순위값,사용목적2순위값,평일일평균레저시간값,주말일평균레저시간값,휴식오락사용비율,취미사용비율,본인계발사용비율,대인관계교제사용비율,기타사용비율,계절,위도,경도,또래인구수
83127,0,2,2,6,0,1,2,30.0,10.0,20.0,40.0,0.0,3,37.8228,128.1555,207692
35029,1,1,4,0,5,1,10,40.0,20.0,10.0,30.0,0.0,0,37.4138,127.5183,1871078
129418,0,3,1,0,5,2,5,20.0,20.0,20.0,40.0,0.0,3,37.5665,126.9780,1476224
158119,1,3,2,5,1,2,2,70.0,0.0,0.0,30.0,0.0,1,37.4138,127.5183,2318045
226141,1,1,1,6,5,3,3,50.0,0.0,20.0,30.0,0.0,0,35.5384,129.3114,136439
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
97617,0,1,2,7,5,2,6,10.0,0.0,90.0,0.0,0.0,0,37.4138,127.5183,1871078
203867,0,4,4,5,9,6,8,50.0,30.0,10.0,10.0,0.0,3,37.5665,126.9780,1301333
21584,0,0,0,5,8,2,2,10.0,0.0,10.0,80.0,0.0,2,35.8714,128.6014,281916
36543,1,0,1,6,9,5,10,60.0,0.0,10.0,30.0,0.0,3,35.1595,126.8526,190483


In [ ]:
# import pandas as pd
# import numpy as np
# from sklearn.model_selection import train_test_split, GridSearchCV
# from sklearn.ensemble import RandomForestClassifier
# from sklearn.metrics import accuracy_score
# from tqdm import tqdm
# from sklearn.model_selection import KFold

# # 데이터 준비 (이 부분은 실제 데이터로 대체하세요)
# # df = pd.DataFrame(...)  # 여기에 실제 데이터를 로드하세요

# X = df.drop('관심레저활동1순위값', axis=1)
# y = df['관심레저활동1순위값']
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=11)

# # tqdm을 적용한 사용자 정의 CV splitter
# class TqdmKFold(KFold):
#     def __init__(self, n_splits=5, shuffle=True, random_state=None):
#         super().__init__(n_splits=n_splits, shuffle=shuffle, random_state=random_state)
    
#     def split(self, X, y=None, groups=None):
#         return tqdm(super().split(X, y, groups), total=self.n_splits, desc="CV Folds")


# # 탐색할 하이퍼파라미터 범위 정의
# param_grid = {
#     'n_estimators': [100, 200, 300],
#     'max_depth': [None, 10, 20, 30],
#     'min_samples_split': [2, 5, 10],
#     'min_samples_leaf': [1, 2, 4],
#     'max_features': ['auto', 'sqrt']
# }

# # 그리드 탐색 객체 생성
# grid_search = GridSearchCV(RandomForestClassifier(random_state=42), 
#                            param_grid, 
#                            cv=TqdmKFold(n_splits=5, shuffle=True, random_state=42), 
#                            n_jobs=-1, 
#                            verbose=0)  # verbose를 0으로 설정하여 GridSearchCV의 기본 출력을 억제

# # 그리드 탐색 수행
# with tqdm(total=len(param_grid['n_estimators']) * len(param_grid['max_depth']) * 
#           len(param_grid['min_samples_split']) * len(param_grid['min_samples_leaf']) * 
#           len(param_grid['max_features']), desc="Parameter Combinations") as pbar:
#     def update(*args, **kwargs):
#         pbar.update(1)
#     grid_search.fit(X_train, y_train, callback=update)

# # 최적의 파라미터와 점수 출력
# print("Best parameters:", grid_search.best_params_)
# print("Best cross-validation score:", grid_search.best_score_)

# # 테스트 세트로 평가
# y_pred = grid_search.predict(X_test)
# accuracy = accuracy_score(y_test, y_pred)
# print(f"Test set accuracy: {accuracy:.4f}")

In [14]:
# X = df.drop('관심레저활동1순위값', axis=1)
# y = df['관심레저활동1순위값']
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=11)

In [29]:
from imblearn.over_sampling import SMOTE
from imblearn.over_sampling import RandomOverSampler
from collections import Counter

smote = SMOTE(random_state=42)
X_resampled, y_resampled = smote.fit_resample(X, y)

print(Counter(y_resampled))
X_train, X_test, y_train, y_test = train_test_split(X_resampled, y_resampled, test_size=0.2, random_state=11)


Counter({12: 3047, 61: 3047, 52: 3047, 44: 3047, 66: 3047, 58: 3047, 77: 3047, 49: 3047, 48: 3047, 15: 3047, 31: 3047, 62: 3047, 75: 3047, 84: 3047, 59: 3047, 53: 3047, 79: 3047, 16: 3047, 29: 3047, 6: 3047, 80: 3047, 63: 3047, 40: 3047, 50: 3047, 5: 3047, 14: 3047, 32: 3047, 3: 3047, 83: 3047, 39: 3047, 10: 3047, 25: 3047, 24: 3047, 76: 3047, 17: 3047, 55: 3047, 73: 3047, 0: 3047, 69: 3047, 30: 3047, 27: 3047, 34: 3047, 78: 3047, 65: 3047, 81: 3047, 70: 3047, 68: 3047, 22: 3047, 51: 3047, 18: 3047, 82: 3047, 43: 3047, 4: 3047, 47: 3047, 57: 3047, 33: 3047, 35: 3047, 13: 3047, 26: 3047, 2: 3047, 74: 3047, 60: 3047, 64: 3047, 71: 3047, 8: 3047, 36: 3047, 37: 3047, 56: 3047, 38: 3047, 11: 3047, 41: 3047, 1: 3047, 67: 3047, 72: 3047, 28: 3047, 19: 3047, 23: 3047, 21: 3047, 45: 3047, 9: 3047, 42: 3047, 46: 3047, 20: 3047, 54: 3047, 7: 3047})


In [33]:
X_train

,성별,연령대,가구소득정도,사용목적1순위값,사용목적2순위값,평일일평균레저시간값,주말일평균레저시간값,휴식오락사용비율,취미사용비율,본인계발사용비율,대인관계교제사용비율,기타사용비율,계절,위도,경도,또래인구수
83127,0,3,3,8,0,1,3,22.624577,7.375423,12.292372,57.707628,0.0,0,36.099869,126.717125,338551
35029,1,2,1,6,6,2,5,48.242683,13.919106,11.757317,26.080894,0.0,1,35.600276,128.787244,369241
129418,0,1,4,3,2,1,3,14.065386,24.065386,20.000000,41.869228,0.0,2,37.413800,127.518300,1937979
158119,0,3,1,5,0,2,5,48.496615,26.879231,10.751693,13.872461,0.0,2,37.413800,127.518300,2318045
226141,0,2,1,7,4,3,3,70.420422,0.000000,29.369367,0.210211,0.0,3,34.848255,126.482273,239910
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
97617,0,1,1,3,7,4,8,27.491209,5.017582,39.964836,27.526373,0.0,2,35.832790,128.238027,282429
203867,0,4,1,1,4,3,4,27.495456,8.747728,16.243183,47.513633,0.0,1,37.456300,126.705200,438663
21584,0,0,0,5,8,2,2,10.000000,0.000000,10.000000,80.000000,0.0,2,35.871400,128.601400,281916
36543,1,0,2,7,6,3,13,20.343769,20.000000,25.000000,34.656231,0.0,2,37.413800,127.518300,1677623


In [32]:
from sklearn.model_selection import GridSearchCV

# 탐색할 하이퍼파라미터 범위 정의
# param_grid = {
#     'n_estimators': [100, 200, 300],
#     'max_depth': [None, 10, 20, 30],
#     'min_samples_split': [2, 5, 10],
#     'min_samples_leaf': [1, 2, 4],
#     'max_features': ['auto', 'sqrt']
# }
param_grid = {'max_depth': [30,],
  'max_features': ['sqrt',],
  'min_samples_leaf': [1],
  'min_samples_split': [2],
  'n_estimators': [300],
}

# 그리드 탐색 객체 생성
grid_search = GridSearchCV(RandomForestClassifier(random_state=42), 
                           param_grid, 
                           cv=5, 
                           n_jobs=-1, 
                           verbose=2)

# 그리드 탐색 수행
grid_search.fit(X_train, y_train)


# model = RandomForestClassifier(n_estimators=100, random_state=42)
# model.fit(X_train, y_train)
y_pred = grid_search.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print(f"정확도: {accuracy:.6f}")

Fitting 5 folds for each of 1 candidates, totalling 5 fits
정확도: 0.881059


In [34]:
grid_search.best_params_, grid_search.best_score_(['accuracy'])

TypeError: 'numpy.float64' object is not callable

In [30]:
rf = RandomForestClassifier(random_state=11)
rf.fit(X_train, y_train)
y_pred = rf.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print(accuracy)

0.8745342574180969


In [26]:
from xgboost import XGBClassifier
xgb = XGBClassifier(random_state=11)
xgb.fit(X_train, y_train)
y_pred = xgb.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print(accuracy)

0.902990405220178


In [ ]:
# import optuna
# from xgboost import XGBClassifier
# from sklearn.model_selection import cross_val_score, train_test_split
# import numpy as np

# # 데이터 분할 (이전에 하지 않았다면)
# X_train, X_test, y_train, y_test = train_test_split(X_resampled, y_resampled, test_size=0.2, random_state=42)

# def objective(trial):
#     params = {
#         'max_depth': trial.suggest_int('max_depth', 1, 9),
#         'learning_rate': trial.suggest_loguniform('learning_rate', 1e-3, 1.0),
#         'n_estimators': trial.suggest_int('n_estimators', 100, 1000),
#         'min_child_weight': trial.suggest_int('min_child_weight', 1, 10),
#         'subsample': trial.suggest_uniform('subsample', 0.5, 1.0),
#         'colsample_bytree': trial.suggest_uniform('colsample_bytree', 0.5, 1.0),
#         'gamma': trial.suggest_loguniform('gamma', 1e-8, 1.0),
#         'reg_alpha': trial.suggest_loguniform('reg_alpha', 1e-8, 1.0),
#         'reg_lambda': trial.suggest_loguniform('reg_lambda', 1e-8, 1.0),
#     }
    
#     model = XGBClassifier(**params, use_label_encoder=False, eval_metric='logloss', random_state=42)
    
#     score = cross_val_score(model, X_train, y_train, cv=5, scoring='accuracy')
    
#     return score.mean()

# # Optuna 연구 생성 및 최적화 실행
# study = optuna.create_study(direction='maximize')
# study.optimize(objective, n_trials=100)

# # 최적의 하이퍼파라미터 및 점수 출력
# print('Best trial:')
# trial = study.best_trial
# print('  Value: {}'.format(trial.value))
# print('  Params: ')
# for key, value in trial.params.items():
#     print('    {}: {}'.format(key, value))

# # 최적의 모델로 학습
# best_params = study.best_params
# best_model = XGBClassifier(**best_params, use_label_encoder=False, eval_metric='logloss', random_state=42)
# best_model.fit(X_train, y_train)

# # 테스트 데이터로 평가
# accuracy = best_model.score(X_test, y_test)
# print(f"Best Model Accuracy on test data: {accuracy}")

# # 특성 중요도 출력 (선택사항)
# feature_importance = best_model.feature_importances_
# for i, importance in enumerate(feature_importance):
#     print(f"Feature {i}: {importance}")